# Lesson 11: FRI Protocol (Commit Phase)

In this lesson, we'll examine the commit phase of the FRI (Fast Reed-Solomon IOP) protocol. This phase creates cryptographic commitments for each protocol layer, allowing the verifier to later check proof correctness.

In [ ]:
import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Tuple
import hashlib

@dataclass
class FRICommitment:
    """Structure for storing FRI commitments."""
    merkle_root: bytes
    layer_values: np.ndarray
    alpha: complex

class FRICommitter:
    """Implements FRI protocol commit phase."""
    
    def __init__(self, initial_values: np.ndarray, num_layers: int):
        self.initial_values = initial_values
        self.num_layers = num_layers
        self.domain_size = len(initial_values)
        
    def create_merkle_root(self, values: np.ndarray) -> bytes:
        """Creates Merkle tree root for values."""
        # Hash each value
        leaves = [hashlib.sha256(str(v).encode()).digest() for v in values]
        
        # Build tree bottom-up
        while len(leaves) > 1:
            next_level = []
            for i in range(0, len(leaves), 2):
                left = leaves[i]
                right = leaves[i + 1] if i + 1 < len(leaves) else left
                parent = hashlib.sha256(left + right).digest()
                next_level.append(parent)
            leaves = next_level
            
        return leaves[0]
    
    def commit_layer(self, values: np.ndarray, alpha: complex) -> FRICommitment:
        """Creates commitment for one FRI layer."""
        merkle_root = self.create_merkle_root(values)
        return FRICommitment(merkle_root, values, alpha)
    
    def generate_commitments(self) -> List[FRICommitment]:
        """Generates sequence of FRI commitments."""
        from lesson_10_fri_mixing import FRIMixer
        
        mixer = FRIMixer(self.domain_size)
        commitments = []
        current_values = self.initial_values
        
        # Create commitment for initial layer
        initial_commitment = self.commit_layer(current_values, 0+0j)
        commitments.append(initial_commitment)
        
        # Create commitments for subsequent layers
        for _ in range(self.num_layers):
            next_values, alpha = mixer.create_fri_layer(current_values)
            commitment = self.commit_layer(next_values, alpha)
            commitments.append(commitment)
            current_values = next_values
            
        return commitments

## Demonstrating Commit Phase

In [ ]:
# Create test data
domain_size = 16
x = np.linspace(0, 2*np.pi, domain_size)
initial_values = np.sin(x) + 1j * np.cos(x)

# Create committer and generate commitments
committer = FRICommitter(initial_values, num_layers=3)
commitments = committer.generate_commitments()

# Visualization
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))

# Plot values in each layer
plt.subplot(2, 1, 1)
for i, commitment in enumerate(commitments):
    plt.plot(np.abs(commitment.layer_values), 
             label=f'Layer {i} (|values|)')
plt.title('Absolute Values in FRI Layers')
plt.xlabel('Index')
plt.ylabel('|Value|')
plt.grid(True)
plt.legend()

# Plot value phases
plt.subplot(2, 1, 2)
for i, commitment in enumerate(commitments):
    plt.plot(np.angle(commitment.layer_values), 
             label=f'Layer {i} (phase)')
plt.title('Value Phases in FRI Layers')
plt.xlabel('Index')
plt.ylabel('Phase (rad)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Print commitment information
print("\nFRI Commitment Information:")
for i, commitment in enumerate(commitments):
    print(f"\nLayer {i}:")
    print(f"Layer size: {len(commitment.layer_values)}")
    print(f"Merkle root: {commitment.merkle_root.hex()[:16]}...")
    if i > 0:
        print(f"Coefficient α: {commitment.alpha:.3f}")

## Analysis of FRI Commit Phase

1. **Commitment Structure**:
   - Each layer has its own commitment
   - Commitments linked through mixing coefficients
   - Layer sizes decrease geometrically

2. **Cryptographic Properties**:
   - Binding: cannot change values after commit
   - Hiding: commitments don't reveal values
   - Efficient verifiability

3. **Optimizations**:
   - Use of Merkle trees
   - Efficient value storage
   - Parallel computations

## Next Steps

1. **Query Phase**:
   - Verifier requests specific values
   - Prover provides Merkle proofs
   - Layer consistency is checked

2. **Integration into STARK**:
   - Commitments used in overall proof
   - Connection with other components
   - Final verification

In the next lesson, we'll examine the query phase of the FRI protocol, where these commitments are used for verification.